### EDA do meu dataset.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import dtale
import math
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from joblib import dump
from joblib import load
from scipy.stats import ranksums, kruskal
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pandas_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
#PYOD
from pyod.models.copod import COPOD
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

joblibs_folder_name = 'Joblib'
joblib_file_name = 'dados_preparados.joblib'
joblib_file_name_pos_exploratorio = 'dados_pos_exploratorio.joblib'

base_data_path = 'Dados'

In [ ]:
sns.set_palette("Accent_r")
sns.set_style("darkgrid")
sns.set(font_scale = 1.5)
a4_dims = (11.7, 8.27)

In [ ]:
def plota_ridge(data, row, hue, y):
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
    # Initialize the FacetGrid object
    pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
    g = sns.FacetGrid(data=data, row=row, hue=hue, aspect=15, height=.9, palette=pal)

    # Draw the densities in a few steps
    g.map(sns.kdeplot, y,
          bw_adjust=.5, clip_on=False,
          fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, y, clip_on=False, color="w", lw=2, bw_adjust=.5)
    g.map(plt.axhline, y=0, lw=2, clip_on=False)


    # Define and use a simple function to label the plot in axes coordinates
    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
                ha="left", va="center", transform=ax.transAxes)


    g.map(label, y)

    # Set the subplots to overlap
    g.fig.subplots_adjust(hspace=-.25)

    # Remove axes details that don't play well with overlap
    g.set_titles("")
    g.set(yticks=[])
    g.despine(bottom=True, left=True)

In [ ]:
def get_pesos_por_classe(dataset, label_feature_categorica, label_feature_numerica):

    #pegando as métricas:menor média dos grupos, menor mediana dos grupos, valor mínimo e valor máximo
    media_classe_minima = dataset.groupby([label_feature_categorica])[label_feature_numerica].describe().sort_values('50%').iloc[0, :][1]
    mediana_classe_minima = dataset.groupby([label_feature_categorica])[label_feature_numerica].describe().sort_values('50%').iloc[0, :][5]
    valor_minimo_classe_minima = dataset.groupby([label_feature_categorica])[label_feature_numerica].describe().sort_values('50%').iloc[0, :][3]
    valor_maximo_classe_minima = dataset.groupby([label_feature_categorica])[label_feature_numerica].describe().sort_values('50%').iloc[0, :][7]
    
    #pegando o percentil 90 e 75 do grupo de menor mediana
    grupo = dataset.groupby([label_feature_categorica])[label_feature_numerica].median().sort_values(ascending=True).reset_index().iloc[0,0]
    valor_percentil_90_classe_minima = dados.groupby(label_feature_categorica)[label_feature_numerica].get_group(grupo).quantile(.9)
    valor_percentil_75_classe_minima = dados.groupby(label_feature_categorica)[label_feature_numerica].get_group(grupo).quantile(.75)
    
    #calculando o peso mínimo
    peso_minimo = 0.80*mediana_classe_minima+0.15*media_classe_minima+0.025*valor_percentil_75_classe_minima+0.025*valor_percentil_90_classe_minima
    
    #calculando para cada categoria
    serie_medias = dataset.groupby([label_feature_categorica])[label_feature_numerica].transform('mean') 
    serie_medianas = dataset.groupby([label_feature_categorica])[label_feature_numerica].transform('median')
    serie_minimos = dataset.groupby([label_feature_categorica])[label_feature_numerica].transform('min')
    serie_percentil_75 = dados.groupby(label_feature_categorica)[label_feature_numerica].transform(lambda x:np.percentile(x, 75))
    serie_percentil_90 = dados.groupby(label_feature_categorica)[label_feature_numerica].transform(lambda x:np.percentile(x, 90))
    serie_maximos = dataset.groupby([label_feature_categorica])[label_feature_numerica].transform('max')
    
    serie_pesos = 0.80*serie_medianas+0.15*serie_medias+0.025*serie_percentil_75+0.025*serie_percentil_90
    #normalizando pelo de menor mediana
    serie_pesos_normalizada = (serie_pesos / peso_minimo) - 1
    #normalizar e realizar esse teste (mixmaxscaler, normalizar entre 0 e 1).
    serie_teste = (serie_medianas/mediana_classe_minima) - 1
    return serie_pesos_normalizada
    

In [ ]:
def convert_ordinal_by_median_order(label_feature_categorica, label_feature_numerica, dataset):
    keys = dataset.groupby([label_feature_categorica])[label_feature_numerica].agg('median').sort_values().index.tolist()
    values = np.arange(1, len(dataset.groupby([label_feature_categorica])[label_feature_numerica].agg('median'))+1)

    return dataset[label_feature_categorica].map(dict(zip(keys,values)))

In [ ]:
def get_pesos_por_classe_mediana(dataset, label_feature_categorica, label_feature_numerica):

    #pegando as métricas:menor mediana dos grupos
    mediana_classe_minima = dataset.groupby([label_feature_categorica])[label_feature_numerica].describe().sort_values('50%').iloc[0, :][5]
    
    #calculando para cada categoria
    serie_medianas = dataset.groupby([label_feature_categorica])[label_feature_numerica].transform('median')
    
    #normalizar e realizar esse teste (mixmaxscaler, normalizar entre 0 e 1).
    serie_teste = (serie_medianas/mediana_classe_minima) - 1
    
    return serie_teste

In [ ]:
#carregando os dados que já foram lidos e minimamente tratados.
dados = load(os.path.join(base_data_path,joblibs_folder_name,joblib_file_name))

In [ ]:
dados.shape

In [ ]:
dados.columns

In [ ]:
dados.head()

In [ ]:
dados.info()

In [ ]:
categorical_features = dados.select_dtypes(include = ['category']).columns
numerical_features = dados.select_dtypes(include = [np.number]).columns

print(f'Qtd features categóricas = {len(categorical_features)}')
print(f'Qtd features numéricas = {len(numerical_features)}')

### Análise exploratória univariada.

In [ ]:
sample = dados.sample(dados.shape[0])
profile = ProfileReport(sample)
profile.set_variable('correlations',
                      {'pearson.threshold':0.8,
                       'spearman.threshold':0.8,
                       'kendall.threshold':0.8,
                       'phi_k.threshold':0.8,
                       'cramers.threshold':0.8
                      })
profile.to_widgets()
#profile.to_file("Exploratorio.html")

Com base na correlação de Phik, iremos dropar a seguinte feature: origem_autorizacao.

In [ ]:
dados.drop(columns=['origem_autorizacao'], inplace=True)

### Partiremos agora para uma análise exploratória entre a variável valor_total e as outras variáveis categóricas do meu dataset.

#### valor_total vs natureza_juridica_entidade_simplificada

In [ ]:
dados.groupby(['natureza_juridica_entidade_simplificada'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
dados.groupby(['natureza_juridica_entidade_simplificada'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='natureza_juridica_entidade_simplificada', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='natureza_juridica_entidade_simplificada', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
#irei manter a mesma codificação, que já me atende.
dados['natureza_juridica_entidade_simplificada'].cat.rename_categories({'1': 1, '2': 2, '3':3}, inplace=True)

#### valor_total vs especialidade_leito

In [ ]:
dados.groupby(['especialidade_leito'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

01-Cirurgia; 02-Obstetrícia; 03-Clínica médica; 04-Pacientes sob cuidados prolongados; 05-Psiquiatria; 06-Tisiologia; 07-Pediatria; 08-Reabilitação; 09-Psiquiatria em hospital-dia. Não foi encontrado justicativa para o código 10

In [ ]:
dados.groupby(['especialidade_leito'])['valor_total'].describe(percentiles=[.25, .5, .75, .9]).sort_values('50%')

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='especialidade_leito', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='especialidade_leito', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
#vou manter do jeito que está, sem nenhuma transformação.
# dados['especialidade_leito'] = convert_ordinal_by_median_order(label_feature_categorica='especialidade_leito',
#                                                          label_feature_numerica='valor_total',
#                                                          dataset=dados)

#### valor_total vs tipo_AIH

In [ ]:
dados.groupby(['tipo_AIH'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

Tipo 1: inicial e Tipo 5: continuidade

In [ ]:
dados.groupby(['tipo_AIH'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='tipo_AIH', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='tipo_AIH', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados['AIH_continuidade'] = dados['tipo_AIH'].apply(lambda x: 1 if x=='5' else 0).astype('category')
dados.drop(columns='tipo_AIH', inplace=True)

In [ ]:
dados.groupby(['AIH_continuidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='AIH_continuidade', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

#### valor_total vs sexo

In [ ]:
dados.groupby(['sexo'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

1 - masculino, 3 - feminino

In [ ]:
dados.groupby(['sexo'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='sexo', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='sexo', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns=['sexo'], inplace=True)

#### valor_total vs gestao_hospitalar

In [ ]:
dados.groupby(['gestao_hospitalar'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

0 - gestão estadual; 1 - gestão plena municipal; 2 - gestão plena estadual; demais: não definida

In [ ]:
dados.groupby(['gestao_hospitalar'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='gestao_hospitalar', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='gestao_hospitalar', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns=['gestao_hospitalar'], inplace=True)

#### valor_total vs obito

In [ ]:
dados.groupby(['obito'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
dados.groupby(['obito'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='obito', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='obito', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
#manter

#### valor_total vs carater_internacao

In [ ]:
dados.groupby(['carater_internacao'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

01-Eletivo; 02-Urgência; 03-Acidente no local de trabalho ou a serviço da empresa; 04-Acidente no trajeto para o trabalho; 05-Outros tipos de acidente de transito; 06-Outros tipos de lesões e envenenamentos por agentes químicos ou físicos

In [ ]:
dados.groupby(['carater_internacao'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='carater_internacao', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='carater_internacao', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
#mantem
#atenção na classe 02: veja o salto de 90% para max.
# dados['carater_internacao'] = convert_ordinal_by_median_order(dataset=dados,
#                                                         label_feature_categorica='carater_internacao',
#                                                         label_feature_numerica='valor_total')

#### valor_total vs complexidade

In [ ]:
dados.groupby(['complexidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

1 - Atenção Básica Complexidade, 2 - Média Complexidade, 3 - Alta Complexidade

In [ ]:
dados.groupby(['complexidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='complexidade', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='complexidade', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados['complexidade_alta'] = dados['complexidade'].apply(lambda x: 1 if x=='03' else 0).astype('category')
dados.drop(columns='complexidade', inplace=True)

In [ ]:
dados.groupby(['complexidade_alta'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='complexidade_alta', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

#### valor_total vs tipo_financiamento

In [ ]:
dados.groupby(['tipo_financiamento'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

01 - Atenção Básica (PAB); 02 - Assistência Farmacêutica; 04- Fundo de Ações Estratégicas e Compensações (FAEC); 05-Incentivo - MAC; 06- Média e Alta Complexidade (MAC); 07 - Vigilância em Saúde; 08 - Gestão do SUS

In [ ]:
dados.groupby(['tipo_financiamento'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='tipo_financiamento', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='tipo_financiamento', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns=['tipo_financiamento'], inplace=True)

#### valor_total vs tipo_unidade

In [ ]:
dados.groupby(['tipo_unidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

 "05"="HOSPITAL GERAL", "07"="HOSPITAL ESPECIALIZADO", "15";"UNIDADE MISTA", "62";"HOSPITAL/DIA - ISOLADO" <br>
 Detalhamento em http://tabnet.datasus.gov.br/cgi/cnes/tipo_estabelecimento.htm#:~:text=Unidade%20Mista%3A%20Unidade%20de%20sa%C3%BAde,de%20interna%C3%A7%C3%A3o%2C%20sob%20administra%C3%A7%C3%A3o%20%C3%BAnica.

In [ ]:
dados.groupby(['tipo_unidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='tipo_unidade', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='tipo_unidade', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados['tipo_unidade_62'] = dados['tipo_unidade'].apply(lambda x: 1 if x=='62' else 0).astype('category')
dados.drop(columns='tipo_unidade', inplace=True)

In [ ]:
dados.groupby(['tipo_unidade_62'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='tipo_unidade_62', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

#### valor_total vs grau_dependencia_entidade

In [ ]:
dados.groupby(['grau_dependencia_entidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

Grau de dependência: 1-Individual, 3-Mantida

In [ ]:
dados.groupby(['grau_dependencia_entidade'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='grau_dependencia_entidade', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='grau_dependencia_entidade', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns='grau_dependencia_entidade', inplace=True)

#### valor_total vs atividade_ensino

In [ ]:
dados.groupby(['atividade_ensino'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
dados.groupby(['atividade_ensino'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='atividade_ensino', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='atividade_ensino', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns='atividade_ensino', inplace=True)

#### valor_total vs porte_hospitalar

In [ ]:
dados.groupby(['porte_hospitalar'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
dados.groupby(['porte_hospitalar'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='porte_hospitalar', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='porte_hospitalar', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
# # vou manter, sem distinção.
# dados['porte_hospitalar_pequeno'] = dados['porte_hospitalar'].apply(lambda x: 1 if x=='PEQUENO' else 0).astype('category')
# dados.drop(columns='porte_hospitalar', inplace=True)

#### valor_total vs municipio_extrema_probreza

In [ ]:
dados.groupby(['municipio_extrema_probreza'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
dados.groupby(['municipio_extrema_probreza'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='municipio_extrema_probreza', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='municipio_extrema_probreza', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns=['municipio_extrema_probreza'], inplace=True)

#### valor_total vs qtd_municipios_regiao_saude

In [ ]:
dados.groupby(['qtd_municipios_regiao_saude'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='qtd_municipios_regiao_saude', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='qtd_municipios_regiao_saude', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns='qtd_municipios_regiao_saude', inplace=True)

#### valor_total vs classe_populacao

In [ ]:
dados.groupby(['classe_populacao'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.boxplot(x='classe_populacao', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.violinplot(x='classe_populacao', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
dados.drop(columns=['classe_populacao'],inplace=True)

#### valor_total vs capitulo_CID

In [ ]:
dados.groupby(['capitulo_CID'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
dados.groupby(['capitulo_CID'])['valor_total'].describe(percentiles=[.25, .5, .75, .9]).sort_values('50%')

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.boxplot(x='capitulo_CID', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.violinplot(x='capitulo_CID', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
# #avaliar fazer uma categorizacao por faixa de valor
# #outras sugestões do Eugênio: tirar por simplicidade OU avaliar manter os scores

# #buscando os CID cujos valores medianos <500, os quais denominarei 1 (baratos)
# cid_valores_medianos_baratos = dados.groupby(['capitulo_CID'])['valor_total'].describe(percentiles=[.25, .5, .75, .9]).sort_values('50%').loc[:,'50%']<500
# cid_valores_medianos_baratos = cid_valores_medianos_baratos[cid_valores_medianos_baratos==True].index

# #buscando os CID cujos valores medianos > 1000, os quais denominarei 3 (caros)
# cid_valores_medianos_caros = dados.groupby(['capitulo_CID'])['valor_total'].describe(percentiles=[.25, .5, .75, .9]).sort_values('50%').loc[:,'50%']>1000
# cid_valores_medianos_caros = cid_valores_medianos_caros[cid_valores_medianos_caros==True].index

# # valor mediano por CID <500 = 1
# # valor mediano por CID > 1000 = 3
# # valor mediano >= 500 OU <= 1000 = 2
# transforma_cid = lambda cap_cid: 1 if cap_cid in cid_valores_medianos_baratos else\
#                           (3 if cap_cid in cid_valores_medianos_caros else 2)  


# dados['CID_valor_mediano'] = dados['capitulo_CID'].map(transforma_cid)
dados.drop(columns=['capitulo_CID'], inplace=True)

#### valor_total vs internacao_com_UTI

In [ ]:
dados.groupby(['internacao_com_UTI'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x='internacao_com_UTI', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(x='internacao_com_UTI', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
#manter

#### valor_total vs reside_local_atendimento

In [ ]:
dados.groupby(['reside_local_atendimento'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.boxplot(x='reside_local_atendimento', y=np.log10(dados.valor_total), data=dados, notch=True)

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.violinplot(x='reside_local_atendimento', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
#dropar
#dúvida: não vejo tanta diferença nas distribuições. caso clássico que devemos dropar!
dados.drop(columns=['reside_local_atendimento'], inplace=True)

#### valor_total vs procedimento_realizado

In [ ]:
dados.groupby(['procedimento_realizado'])['valor_total'].describe().sort_values('50%')

In [ ]:
# #dúvida: como mostrar para o algoritmo tamanha heterogeneidade das classes dos procedimentos?

# dados['procedimento_realizado_peso'] = get_pesos_por_classe(dataset=dados,
#                                                   label_feature_categorica='procedimento_realizado',
#                                                   label_feature_numerica='valor_total')

#### valor_total vs idade

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.scatterplot(data=dados, x='idade', y=np.log10(dados.valor_total))

In [ ]:
dados['idade_categorizada'] = pd.cut(x=dados['idade'], bins=[0,19,24,28,34,39,44,49,54,59,101], 
                                     right=False, labels=None, retbins=False, precision=3, 
                                     include_lowest=True, duplicates='raise', ordered=True)

In [ ]:
dados.groupby(['idade_categorizada'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
#colocar por ordem olhando para  os bins de idade.
dados.groupby(['idade_categorizada'])['valor_total'].describe(percentiles=[.25, .5, .75, .9])

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
plt.xticks(rotation=45)
sns.violinplot(x='idade_categorizada', y=np.log10(dados.valor_total), data=dados, notch=True, inner='box')

In [ ]:
# por simplificação, irei alterar a feature para uma flag idade_maior_44, pois não vejo grandes diferenças entre os grupos,
# mas a partir do grupos acima de 44 anos, temos o percentil 75 com valor maior que 1000.
dados['idade_maior_44'] = dados['idade'].map(lambda x: 1 if x>=44 else 0)
dados.drop(columns=['idade', 'idade_categorizada'], inplace=True)

In [ ]:
sample = dados.sample(dados.shape[0])
profile = ProfileReport(sample)
profile.set_variable('correlations',
                      {'pearson.threshold':0.8,
                       'spearman.threshold':0.8,
                       'kendall.threshold':0.8,
                       'phi_k.threshold':0.8,
                       'cramers.threshold':0.8
                      })
profile.to_widgets()
#profile.to_file("Exploratorio.html")

In [ ]:
#Salva o Dataframe com os dados transformados na exploratória no disco. Caso já esteja salvo, apaga-o antes.
if os.path.exists(os.path.join(base_data_path,joblibs_folder_name,joblib_file_name_pos_exploratorio)):
    os.remove(os.path.join(base_data_path,joblibs_folder_name,joblib_file_name_pos_exploratorio))
dump(dados, os.path.join(base_data_path,joblibs_folder_name,joblib_file_name_pos_exploratorio))